<a href="https://colab.research.google.com/github/lapatradaa/shap/blob/main/model_frac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#เรียกใช้ CSV

In [1]:
import pandas as pd
import shutil

In [30]:
#เชื่อม google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Imports

In [3]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import os
import glob
import shutil
import sys
import numpy as np
from skimage.io import imread
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline
import datetime

In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

#กำหนดค่าพารามิเตอร์


In [5]:
epochs = 250

In [40]:
batch_size = 16
width = 224 # จตุรัส โมเตลจะเรียนรู้ภาพได้ดีที่สุด
height = 224
epochs = 250 # รอบในการเทรน
dropout_rate = 0.2
input_shape = (height, width, 3)

#Clone efficientnet repo


In [7]:
#ดึงข้อมูลใน Github มาใช้
import os
%cd /content
if not os.path.isdir("efficientnet_keras_transfer_learning"):
 !git clone https://github.com/Wanita-8943/efficientnet_keras_transfer_learning
%cd efficientnet_keras_transfer_learning/


/content
Cloning into 'efficientnet_keras_transfer_learning'...
remote: Enumerating objects: 1109, done.
remote: Counting objects: 100% (272/272), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 1109 (delta 137), reused 241 (delta 121), pack-reused 837 (from 1)
Receiving objects: 100% (1109/1109), 14.10 MiB | 20.99 MiB/s, done.
Resolving deltas: 100% (634/634), done.
/content/efficientnet_keras_transfer_learning


In [8]:
# Options: EfficientNetB0, EfficientNetB1, EfficientNetB2, EfficientNetB3
# Higher the number, the more complex the model is.
from efficientnet import EfficientNetB0 as Net
from efficientnet import center_crop_and_resize, preprocess_input

In [12]:
# loading pretrained conv base model
# โหลดโมเดล มาโดยตัด output ของโมเดลออก เเต่ยังใช้ input อันเดิม
# เเละโหลด weight ของโมเดล มาด้วยที่ชื่อว่า imagenet
conv_base = Net(weights='imagenet', include_top=False) # convolution base = base ที่เอาไว้เรียนแบบยังไม่ปรับ

In [13]:
conv_base.summary() #ดู Summary

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_65 (Conv2D)        │ (None, 112, 112, 32)   │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_49    │ (None, 112, 112, 32)   │            128 │ conv2d_65[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ swish_49 (Swish)          │ (None, 112, 112, 32)   │              0 │ batch_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ depthwise_conv2d_16       │ (None, 112, 112, 32)   │            288 │ swish_49[0][0]         │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_50    │ (None, 112, 112, 32)   │            128 │ depthwise_conv2d_16[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ swish_50 (Swish)          │ (None, 112, 112, 32)   │              0 │ batch_normalization_5… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_16 (Lambda)        │ (None, 1, 1, 32)       │              0 │ swish_50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_66 (Conv2D)        │ (None, 1, 1, 8)        │            264 │ lambda_16[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ swish_51 (Swish)          │ (None, 1, 1, 8)        │              0 │ conv2d_66[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_67 (Conv2D)        │ (None, 1, 1, 32)       │            288 │ swish_51[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_16             │ (None, 1, 1, 32)       │              0 │ conv2d_67[0][0]        │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply_16 (Multiply)    │ (None, 112, 112, 32)   │              0 │ activation_16[0][0],   │
│                           │                        │                │ swish_50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_68 (Conv2D)        │ (None, 112, 112, 16)   │            512 │ multiply_16[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_51    │ (None, 112, 112, 16)   │             64 │ conv2d_68[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_69 (Conv2D)        │ (None, 112, 112, 96)   │          1,536 │ batch_normalization_5… │
├──────────────────────

 Total params: 4,049,564 (15.45 MB)

 Trainable params: 4,007,548 (15.29 MB)

 Non-trainable params: 42,016 (164.12 KB)

In [14]:
# ฟังก์ชัน ReLU ทำการปรับค่า x โดยให้เป็น 0 หาก x น้อยกว่า 0 และให้เป็น  x หาก x มากกว่าหรือเท่ากับ 0 นั่นหมายความว่า ReLU จะ "กระตุ้น" ค่าที่มากกว่าหรือเท่ากับ 0 ให้ผ่านไป และจะปรับค่าที่น้อยกว่า 0 เป็น 0 นั่นเอง

In [15]:
# create new model with a new classification layer
x = conv_base.output
global_average_layer = layers.GlobalAveragePooling2D(name = 'head_pooling')(x) # Pooling ข้อมูลที่ออกมาจาก conv_base ซึ่งช่วยในการลดมิติของข้อมูล
# add 1 layer
dropout_layer_1 = layers.BatchNormalization(name = 'batchNormalization')(global_average_layer)
dropout_layer_2 = layers.Dropout(0.20,name = 'head_dropout')(dropout_layer_1)
dense_1 = layers.Dense(64, activation='relu',name = 'pred_dense_1')(dropout_layer_2)
dense_2 = layers.Dense(32, activation='relu',name = 'pred_dense_2')(dense_1)
prediction_gender = layers.Dense(1, activation='sigmoid',name = 'prediction_gender')(dense_2)

In [16]:
### lastlayer
model = models.Model(inputs= conv_base.input, outputs=prediction_gender, name = 'EffNet_Classification')

In [17]:
print('This is the number of trainable layers '
      'of the conv base:', len(conv_base.trainable_weights))

print('This is the number of trainable layers '
      'of the model:', len(model.trainable_weights))

This is the number of trainable layers of the conv base: 211
This is the number of trainable layers of the model: 219


In [18]:
model.summary() #ดู Summary

Model: "EffNet_Classification"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_65 (Conv2D)        │ (None, 112, 112, 32)   │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_49    │ (None, 112, 112, 32)   │            128 │ conv2d_65[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ swish_49 (Swish)          │ (None, 112, 112, 32)   │              0 │ batch_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ depthwise_conv2d_16       │ (None, 112, 112, 32)   │            288 │ swish_49[0][0]         │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_50    │ (None, 112, 112, 32)   │            128 │ depthwise_conv2d_16[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ swish_50 (Swish)          │ (None, 112, 112, 32)   │              0 │ batch_normalization_5… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_16 (Lambda)        │ (None, 1, 1, 32)       │              0 │ swish_50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_66 (Conv2D)        │ (None, 1, 1, 8)        │            264 │ lambda_16[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ swish_51 (Swish)          │ (None, 1, 1, 8)        │              0 │ conv2d_66[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_67 (Conv2D)        │ (None, 1, 1, 32)       │            288 │ swish_51[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_16             │ (None, 1, 1, 32)       │              0 │ conv2d_67[0][0]        │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply_16 (Multiply)    │ (None, 112, 112, 32)   │              0 │ activation_16[0][0],   │
│                           │                        │                │ swish_50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_68 (Conv2D)        │ (None, 112, 112, 16)   │            512 │ multiply_16[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_51    │ (None, 112, 112, 16)   │             64 │ conv2d_68[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_69 (Conv2D)        │ (None, 112, 112, 96)   │          1,536 │ batch_normalization_5… │
├──────────────────────

 Total params: 4,138,781 (15.79 MB)

 Trainable params: 4,094,205 (15.62 MB)

 Non-trainable params: 44,576 (174.12 KB)

In [19]:
##Freeze
print('This is the number of trainable layers '
          'before freezing the conv base:', len(model.trainable_weights))
for layer in conv_base.layers:
    layer.trainable = False
print('This is the number of trainable layers '
          'after freezing the conv base:', len(model.trainable_weights))
print('-'*80)
model.summary()

This is the number of trainable layers before freezing the conv base: 219
This is the number of trainable layers after freezing the conv base: 8
--------------------------------------------------------------------------------


Model: "EffNet_Classification"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_65 (Conv2D)        │ (None, 112, 112, 32)   │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_49    │ (None, 112, 112, 32)   │            128 │ conv2d_65[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ swish_49 (Swish)          │ (None, 112, 112, 32)   │              0 │ batch_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ depthwise_conv2d_16       │ (None, 112, 112, 32)   │            288 │ swish_49[0][0]         │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_50    │ (None, 112, 112, 32)   │            128 │ depthwise_conv2d_16[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ swish_50 (Swish)          │ (None, 112, 112, 32)   │              0 │ batch_normalization_5… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_16 (Lambda)        │ (None, 1, 1, 32)       │              0 │ swish_50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_66 (Conv2D)        │ (None, 1, 1, 8)        │            264 │ lambda_16[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ swish_51 (Swish)          │ (None, 1, 1, 8)        │              0 │ conv2d_66[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_67 (Conv2D)        │ (None, 1, 1, 32)       │            288 │ swish_51[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_16             │ (None, 1, 1, 32)       │              0 │ conv2d_67[0][0]        │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multiply_16 (Multiply)    │ (None, 112, 112, 32)   │              0 │ activation_16[0][0],   │
│                           │                        │                │ swish_50[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_68 (Conv2D)        │ (None, 112, 112, 16)   │            512 │ multiply_16[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_51    │ (None, 112, 112, 16)   │             64 │ conv2d_68[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_69 (Conv2D)        │ (None, 112, 112, 96)   │          1,536 │ batch_normalization_5… │
├──────────────────────

 Total params: 4,138,781 (15.79 MB)

 Trainable params: 86,657 (338.50 KB)

 Non-trainable params: 4,052,124 (15.46 MB)

#สร้างโฟลเดอร์ Train Valodation และ Test

In [31]:
df = pd.read_csv (r'/content/gdrive/MyDrive/dataset_summary_fractued.csv')
df

,Folder_type,Path_Name,Filename,Class
0,train,/content/gdrive/MyDrive/dent_fractured/train/F...,62_png.rf.cdf22df307e37ccc867c8a2349b168da.jpg,0
1,train,/content/gdrive/MyDrive/dent_fractured/train/F...,1-AI-F_jpeg.rf.3191e3524b6fb122854b19cba0eec2e...,0
2,train,/content/gdrive/MyDrive/dent_fractured/train/F...,13_png.rf.a243e260538c236bd11c1350d8594ecc.jpg,0
3,train,/content/gdrive/MyDrive/dent_fractured/train/F...,25_png.rf.f01f853615acefea71b1d1071a70e47c.jpg,0
4,train,/content/gdrive/MyDrive/dent_fractured/train/F...,106_png.rf.49f76c842ed3a483763f17f06627b1eb.jpg,0
...,...,...,...,...
1846,valid,/content/gdrive/MyDrive/dent_fractured/valid/U...,114_JPG.rf.ecf04336fbf6a45c423ad99420cbe742.jpg,1
1847,valid,/content/gdrive/MyDrive/dent_fractured/valid/U...,792_JPG.rf.e225292a442d83024ad5043946de68d7.jpg,1
1848,valid,/content/gdrive/MyDrive/dent_fractured/valid/U...,389_JPG.rf.d217b2e47518d0db8344c16111737447.jpg,1
1849,valid,/content/gdrive/MyDrive/dent_fractured/valid/U...,382_JPG.rf.0160b5d2bbe0279be4dcc86c36540417.jpg,1


In [32]:
valid = df[df['Folder_type'] == 'valid']
valid


,Folder_type,Path_Name,Filename,Class
1617,valid,/content/gdrive/MyDrive/dent_fractured/valid/F...,111_png.rf.86c57341a35d626b0d3238551fa628e7.jpg,0
1618,valid,/content/gdrive/MyDrive/dent_fractured/valid/F...,83_png.rf.a6ccab522a69d0a0d585277f4fa550b7.jpg,0
1619,valid,/content/gdrive/MyDrive/dent_fractured/valid/F...,8_png.rf.6c600ea482bcf34a115eca0c1efbe2d7.jpg,0
1620,valid,/content/gdrive/MyDrive/dent_fractured/valid/F...,30_png.rf.1700c0c853739169fd7ae75c904e9761.jpg,0
1621,valid,/content/gdrive/MyDrive/dent_fractured/valid/F...,18-AI-F_jpeg.rf.3d7e3da5846a8f4c7f671fd3ee6a44...,0
...,...,...,...,...
1846,valid,/content/gdrive/MyDrive/dent_fractured/valid/U...,114_JPG.rf.ecf04336fbf6a45c423ad99420cbe742.jpg,1
1847,valid,/content/gdrive/MyDrive/dent_fractured/valid/U...,792_JPG.rf.e225292a442d83024ad5043946de68d7.jpg,1
1848,valid,/content/gdrive/MyDrive/dent_fractured/valid/U...,389_JPG.rf.d217b2e47518d0db8344c16111737447.jpg,1
1849,valid,/content/gdrive/MyDrive/dent_fractured/valid/U...,382_JPG.rf.0160b5d2bbe0279be4dcc86c36540417.jpg,1


In [33]:
train = df[df['Folder_type'] == 'train']
train

,Folder_type,Path_Name,Filename,Class
0,train,/content/gdrive/MyDrive/dent_fractured/train/F...,62_png.rf.cdf22df307e37ccc867c8a2349b168da.jpg,0
1,train,/content/gdrive/MyDrive/dent_fractured/train/F...,1-AI-F_jpeg.rf.3191e3524b6fb122854b19cba0eec2e...,0
2,train,/content/gdrive/MyDrive/dent_fractured/train/F...,13_png.rf.a243e260538c236bd11c1350d8594ecc.jpg,0
3,train,/content/gdrive/MyDrive/dent_fractured/train/F...,25_png.rf.f01f853615acefea71b1d1071a70e47c.jpg,0
4,train,/content/gdrive/MyDrive/dent_fractured/train/F...,106_png.rf.49f76c842ed3a483763f17f06627b1eb.jpg,0
...,...,...,...,...
1488,train,/content/gdrive/MyDrive/dent_fractured/train/U...,450_JPG.rf.08e109dbd1d25228c9eabb6e8c8a428e.jpg,1
1489,train,/content/gdrive/MyDrive/dent_fractured/train/U...,842_JPG.rf.9ea4687b37328c2547e97ab9db65bb80.jpg,1
1490,train,/content/gdrive/MyDrive/dent_fractured/train/U...,15_JPG.rf.76f984dac14e2b335d3f5827b9af6127.jpg,1
1491,train,/content/gdrive/MyDrive/dent_fractured/train/U...,1032_JPG.rf.46b3da268e44925c2255949f403ec798.jpg,1


In [34]:

DATA_PATH = "/content/gdrive/MyDrive/dent_fractured"
os.chdir(DATA_PATH)
#train_dir = os.path.join(DATA_PATH, 'train')
#print(train_dir)
#validation_dir = os.path.join(DATA_PATH, 'validation')
#print(validation_dir)


#Train

In [35]:
train['Path_Name'] = train['Path_Name'].str.replace('/content/gdrive/MyDrive/dent_fractured/', '')


<ipython-input-35-25c1ac83a899>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Path_Name'] = train['Path_Name'].str.replace('/content/gdrive/MyDrive/dent_fractured/', '')


In [36]:
DATA_PATH = '/content/gdrive/MyDrive/dent_fractured'


In [37]:
for path in train['Path_Name']:
    full_path = os.path.join(DATA_PATH, path)
    if not os.path.exists(full_path):
        print(f"Invalid file path: {full_path}")


In [38]:
# Train ด้วย ImageDataGenerator ของ Keras ซึ่งจะเพิ่มข้อมูลเสริมระหว่างการฝึกเพื่อลดโอกาสเกิด overfitting
#overfitting เกิดจากข้อมูลที่ซับซ้อนกันเกินไป
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255, #โมเดลส่วนใหญ่ต้องใช้ RGB ในช่วง 0–1
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      # horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

In [39]:
train_generator = train_datagen.flow_from_dataframe(
        dataframe = train,
        directory = DATA_PATH,
        x_col = 'Path_Name',
        y_col = 'Class',
        class_mode = 'raw',
        color_mode= 'rgb',
        target_size=(height, width),
        batch_size=batch_size)

validation_generator = test_datagen.flow_from_dataframe(
        dataframe = valid,
        directory = DATA_PATH,
        x_col = 'Path_Name',
        y_col = 'Class',
        class_mode = 'raw',
        color_mode= 'rgb',
        target_size=(height, width),
        batch_size=batch_size)


Found 0 validated image filenames.
Found 0 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 1493 invalid image filename(s) in x_col="Path_Name". These filename(s) will be ignored.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 234 invalid image filename(s) in x_col="Path_Name". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
model.compile(loss='binary_crossentropy',
          optimizer=Adam(learning_rate= 1e-4),
          metrics=['accuracy'])

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch= len(train) //batch_size,
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps= len(valid) //batch_size,
      verbose=1,
      use_multiprocessing=True,
      workers=4)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']


epochs_x = range(len(loss))



# plt.plot(epochs_x, loss, 'go', label='Training loss')
plt.plot(epochs_x, loss, 'o', color ='darkviolet', label='Training loss')
plt.plot(epochs_x, val_loss, 'k', label='Validation loss')
plt.title('Training and validation loss of Gender Prediction')
plt.legend()

# plt.plot(epochs_x, mse, 'go', label='Training MSE')
plt.figure()
plt.plot(epochs_x, acc, 'o', color ='darkviolet', label='Training accuracy')
plt.plot(epochs_x, val_acc, 'k', label='Validation accuracy')
plt.title('Training and validation Accuracy of Gender Prediction')
plt.legend()



plt.show()

In [ ]:
epochs_x

#Download the model


In [ ]:
os.makedirs('/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Model', exist_ok=True)
model.save('/content/gdrive/MyDrive/Tooth_Shap_GPT/Deep_tooth/Model/Unflipped_Classification_Gender(7-23)/1_Class_1e-4_250_Freeze.h5')